### What is the Segmentation Image? (phân vùng ảnh)

Trong lĩnh vực thị giác máy tính (computer vision), phân vùng ảnh là một quá trình chia một bức ảnh số thành nhiều phần khác nhau (tập hợp các điểm ảnh, hay có thể gọi là superpixels). Mục tiêu của phân vùng ảnh là để đơn giản hóa và hoặc thay đổi biểu diễn của một tấm ảnh vào điều gì đó có ý nghĩa hơn và dễ dàng để phân tích.

Một vài ứng dụng cụ thể trong phân vùng ảnh:

+ Hình ảnh y tế
+ Nhận dạng đối tượng
    + Phát hiện đi bộ
    + Phát hiện khuôn mặt
    + Phát hiện đèn dừng xe
+ Xác định vị trí đối tượng trong ảnh vệ tinh
+ Một số nhiệm vụ nhận dạng
    + Nhận dạng khuôn mặt
    + Nhận dạng vân tay
    + Nhận dạng mắt
+ Hệ thống giám sát giao thông
+ Camera giám sát an ninh

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/03/instance_segmentation_example.jpg)

Nguồn: cs231n.stanford.edu

### How to work?

Khác với bài toán object detection là tìm một box bao quanh một đối tượng. Nên vấn đề là box có thể có phần dư, và một khái niệm mới ra đời để khắc phục vấn đề cho bài toán object detection là segmentation image. 

Bằng việc xem xét một hình ảnh gồm có nhiều đối tượng, một đối tượng là một khu vực trong ảnh, bằng cách cố gắng xác định xem 1 pixel trong ảnh đang thuộc vùng của đối tượng nào. Khi đó bài toán thuộc bài toán classification điểm ảnh vào n vùng.

### Tại sao cần Segmentation Image?

Trong mô hình đang cố phân loại các dựa trên cấp độ pixel vì vậy mà khả năng chính xác khi phân khúc đem đến lợi thế so với các mô hình trong bài toán object detection dừng ở cấp độ bouding box. 

Ví dụ:

Ung thư từ lâu đã là một căn bệnh chết người. Ngay cả trong thời đại tiến bộ công nghệ ngày nay, ung thư có thể gây tử vong nếu chúng ta không xác định nó ở giai đoạn đầu. Phát hiện (các) tế bào ung thư càng nhanh càng tốt có khả năng cứu sống hàng triệu người.

Hình dạng của các tế bào ung thư đóng một vai trò quan trọng trong việc xác định mức độ nghiêm trọng của ung thư. Bạn có thể đã đặt các mảnh lại với nhau - phát hiện đối tượng sẽ không hữu ích ở đây. Chúng tôi sẽ chỉ tạo các hộp giới hạn sẽ không giúp chúng tôi xác định hình dạng của các ô.

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/03/cancer-cell-segmentation.png)




Các thuật toán tốt cho Segmentation Image:

* [1]: Rethinking Atrous Convolution for Semantic Image Segmentation #3 best model for Semantic Segmentation on PASCAL VOC 2012 test

* [2]: U-net

* [3]: SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation

# Khám phá các thuật toán cho phân khúc hình ảnh có kết quả tốt ở thời điểm hiện tại năm 2020 

### U-Net

U-net được phát triển bởi Olaf Ronneberger et al. cho phân đoạn hình ảnh y tế sinh học. Kiến trúc chứa hai đường dẫn. Đường dẫn đầu tiên là đường dẫn co lại (còn được gọi là bộ mã hóa) được sử dụng để chụp bối cảnh trong ảnh. Bộ mã hóa chỉ là một chồng truyền thống của các lớp gộp và tối đa. Đường dẫn thứ hai là đường dẫn mở rộng đối xứng (còn được gọi là bộ giải mã) được sử dụng để cho phép nội địa hóa chính xác bằng cách sử dụng các cấu trúc chuyển vị. Do đó, nó là một mạng chập hoàn toàn từ đầu đến cuối (FCN), tức là nó chỉ chứa các lớp Convolutional và không chứa bất kỳ lớp Dense nào vì nó có thể chấp nhận hình ảnh có kích thước bất kỳ.


![](https://miro.medium.com/max/1280/1*OkUrpDD6I0FpugA_bbYBJQ.png)

![](https://miro.medium.com/max/1632/1*yzbjioOqZDYbO6yHMVpXVQ.jpeg)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model



### Fast Fully-connected network (FastFCN)

### Gated-SCNN

### DeepLab

### Mask R-CNN


### Loss Function in Image Segmentation 

![](https://miro.medium.com/max/1400/1*dWBAei84Ld3HtZPx257tHQ.png)

# NHÓM LIÊN QUAN ENTROPY

* Cross-entropy

Với mục tiêu là phân loại các pixel vào các đối tượng khác nhau hay là vào các vùng khác nhau chúng ta sử dụng hàm mất mát entroy.

Entropy là khái niệm đưa ra trong việc tính toán số bít biểu diễn thông tin nhỏ nhất giúp truyền tin đi mà không bị mất mát thông tin.

Có thể đọc thêm ví dụ: https://forum.machinelearningcoban.com/t/hieu-ve-entropy/4443 để đọc thêm ví dụ 

Giả sử ta có 8 kiểu tin nhắn khác nhau, mỗi 1 loại xuất hiện với xác xuất bằng nhau (p=1/8 = 12.5%).
Chẳng hạn bạn có n tin cần truyền đi, mỗi bit truyền đi có thể mã hóa được cho 2 tin, vậy để mã hóa toàn bộ n tin cần truyền đi ta cần $\log_2 n$. Trong trường hợp trên chuyển 8 kiểu tin thì cần 3 bit để mã hóa.

Hay với 1 loại tin nhắn xuất hiện 1 trong N lần gửi tin, công thức bên trên mô tả kích cỡ tối thiểu cần thiết để mã hóa. Và vì P =1/N là xác xuất của loại tin nhắn đó, phương trình có thể viết lại thành:

![](https://forum.machinelearningcoban.com/uploads/default/original/2X/d/d887c987e408b7c41ff25fe6240d72c126f2e60d.png)

Kết hợp với công thức tính kích cỡ trung bình đã biết ở phần trước, ta được kích thước mã hóa trung bình nhỏ nhất - đây chính là entropy:

![](https://forum.machinelearningcoban.com/uploads/default/original/2X/6/6a9b97d0dc96135ef2b4ba0a1298792add711730.png)


Tính chất entropy: 

+ Entropy là lượng bit trung bình tối thiểu để mã hóa thông tin khi ta biết phân bố các loại thông tin trong đó.
+ Entropy biểu thị cho sự hỗn độn, độ bất định, độ phức tạp của thông tin.
+ Thông tin càng phức tạp càng entropy càng cao (hay công sức mã hóa lớn).
+ Entropy là nền tảng trong việc công thức cross-entropy đo lường sai khác giữa các phân bố xác suất.
+ Entropy nhạy cảm với thay đổi xác suất nhỏ, khi 2 phân bố càng giống nhau thì entropy càng giống nhau và ngược lại
+ Entropy thấp đồng nghĩa với việc hầu hết các lần nhận thông tin, ta có thể dự đoán dễ hơn, ít bất ngờ hơn, ít bất ổn hơn và ít thông tin hơn.

Vậy áp dụng trong bài toán phân loại nói chung ra sao (bới segment mình đang quy nó về một bài toán phân loại các pixel):

Trước hết hãy xem xét entropy đạt giá trị max khi nào nhé:

Xét bài toán có 2 lớp cho việc segment người thì 1 pixel có thể nhận 2 giá trị: 1. nằm trong vùng người, 2. nằm vùng nền.

Với từng pixel j giả sử xác xuất phân loại nó vào object A là p (0<=p<=1), lớp b là 1 - p 

Như vậy entropy của sự kiện pixel j nằm trong object A là: 

H = $-{(p\log(p) + (1 - p)\log(1 - p))}$

Trong trường hợp có nhiều lớp thì:
H = $-\sum_{c=1}^My_{o,c}\log(p_{o,c})$


H đạt giá trị max khi nào, và min khi nào?????

H đạt giá trị max khi: 

Với -oo <= log(p) hoặc log(1-p) <= 0

Ta tính H'(p) = -[ (log(p) + p*p) + ( -log(1-p)  + (1-p)(p-1) ) ]


Hàm số đạt giá trị lớn nhất khi p = 1/2 
Khi đó entropy mức độ đống nhất dữ liệu tại mỗi lớp là kém nhất.  Chính lẻ đó nến p càng lớn thì H càng nhỏ. 

Thử lại: p = 0.9 => H = 0.46, p = 0.7 => H = 0.88. Xác xuất rơi vào một nhóm càng lớn, thì nhóm còn lại nhỏ, tức entropy thấp đi.

Chính lẻ đó đây là một trong hàm được sử dụng trong bài toán phân loại.


CE sẽ không hoạt động tốt trong bài toán imblance nhưng trong TH object chỉ chứa 10% pixel và nền 90% pixel. Nên có vài phương pháp cải tiến chỉnh sửa CE tiêu chuẩn:

* Weight CE (WCE)

Do sự mất câng bằng dữ liệu nên tư tưởng chính hàm loss này là tăng trọng số tức tăng loss lên của TH phân loại sai của lớp thiểu số với beta > 1, và giảm loss của TH phân loại lớp đa số khi beta < 1.

Chẳng hạn việc chỉ có 10% pixel thuộc object và 90% pixel còn lại là nền thì loss khi một pixel được phân loại sai tăng lên beta lần (với beata > 1) và ngược lại với pixel nền bị phân loại sai.

Code hàm loss triểu khai trong keras.

In [ ]:
def weighted_cross_entropy(beta):
    def convert_to_logits(y_pred):
        # see https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/keras/backend.py#L3525
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

        return tf.log(y_pred / (1 - y_pred))

    def loss(y_true, y_pred):
        y_pred = convert_to_logits(y_pred)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, targets=y_true, pos_weight=beta)
        # or reduce_sum and/or axis=-1
        return tf.reduce_mean(loss)

    return loss


* Balanced cross entropy 

Nó tương tự như WCE nhưng điểm khác khi mà WCE chỉ xem xét trọng số cho các mẫu tích cực, thì ở đây BCE xem xét cả 2 phần tích cực và tiêu cực.

$\text{BCE}\left(p, \hat{p}\right) = -\left(\beta p \log\left(\hat{p}\right) + (1 - \beta)(1-p) \log\left(1 - \hat{p}\right)\right)$




In [ ]:
def balanced_cross_entropy(beta):
    def convert_to_logits(y_pred):
        
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

        return tf.log(y_pred / (1 - y_pred))

    def loss(y_true, y_pred):
        y_pred = convert_to_logits(y_pred)
        pos_weight = beta / (1 - beta)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, targets=y_true, pos_weight=pos_weight)

        # or reduce_sum and/or axis=-1
        return tf.reduce_mean(loss * (1 - beta))

    return loss


Nãy h nhìn thấy hàm convert_to_logits dùng trong cả WCE và BCE. Vậy hàm này có tác dụng gì. Đầu ra dự đoán của chúng ta thường được đi qua một hàm sigmoid. Chúng ta cần đảo ngược quá trình đó lại để có hiệu quả trong việc tính toán hàm loss. 

Công thức hàm sigmoid:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/9537e778e229470d85a68ee0b099c08298a1a3f6)

Và đảo ngược lại tức tìm x = $\log\left(\frac{\hat{y}}{1 - \hat{y}}\right) = \log\left(\frac{\frac{1}{1 + e^{-x}}}{1 - \frac{1}{1 + e^{-x}}}\right) = x$

Nhưng tại sao lại có hàm: 

tf.clip_by_value 

Chức năng hàm này: Đưa ra một tenxơ t, thao tác này trả về một tenxơ cùng loại và hình dạng như tvới các giá trị của nó được cắt clip_value_minvà clip_value_max. Bất kỳ giá trị nhỏ hơn clip_value_minđược đặt thành clip_value_min. Bất kỳ giá trị lớn hơn clip_value_maxđược đặt thành clip_value_max.

Tại sao cần làm vậy hãy để ý quá trình tính như vậy. Hãy quan sát công thức tìm x:

1-pred mẫu số. Điều gì xảy ra nếu pred = 1. Chính lẻ đó nên nta chỉ sử dụng 1 giá trị tiến sát tới 1. Trong TH này là 1 - (1e-7).

Còn tại sao k dùng pred = 0, đơn giản log(0) có tồn tại hay không hahahaha. 

* Focal Loss

![](https://i1.wp.com/neptune.ai/wp-content/uploads/focal-loss.png?w=237&ssl=1)

Focal loss được sử dụng cải tiến Cross Entropy những điểm:

Khi một mẫu bị phân loại sai và pt nhỏ, modulating factor(1-pt) gần 1 và loss sẽ không bị ảnh hưởng. Còn khi pt tiến tới 1, tức các trường hợp được phân loại tốt, moduling factor sẽ tiến tới 0 và hàm loss trong trường hợp này sẽ bị giảm trọng số xuống.

Focal loss giảm ảnh hưởng nhiều của các vùng dễ phân biệt(“easy classify”) (bên phải hình dưới). Trong bài toán segmentation với những objet nhỏ số lượng pixel ít hơn nhiều so với việc pixel nền. Nên việc mà sự đóng góp loss của phân loại pixel trong object này rất nhỏ so với sự đóng góp của việc phân loại pixel nền. 

Vì vậy, với số lượng rất nhiều, thì lúc training mạng loss của vùng khó phân biệt sẽ đóng góp rất nhỏ trong loss chung của mô hình, trong khi loss của FL lại giảm sự đóng góp của các vùng dễ phân biệt và giữ nguyên loss của các các vùng khó phân biệt. Việc này được thể hiện ở công thức trên. Hệ số gamma được gọi là “focusing parameter” thể hiện “tập trung” vào các vùng khó phân biệt, nếu gamma càng lớn thì loss ở vùng dễ phân biệt sẽ càng nhỏ và loss của càng vùng khó phân biệt sẽ đóng góp nhiều hơn vào tổng loss chung của mô hình.

$gama$ là biến số có thể điều chỉnh được (focusing parameter) nằm trong khoảng giá trị [0, 5].

![](https://forum.machinelearningcoban.com/uploads/default/original/2X/6/6dd288377b8d4f68ea85ca34df9af25057e69fd5.png)



In [ ]:
def focal_loss(alpha=0.25, gamma=2):
    def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    
    return (tf.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b 

    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        logits = tf.log(y_pred / (1 - y_pred))

        loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)

        # or reduce_sum and/or axis=-1
        return tf.reduce_mean(loss)

    return loss

# NHÓM LIÊN QUAN BIỆN PHÁP CHỒNG CHÉO
Khi sử dụng mất entropy chéo, phân phối thống kê của nhãn đóng vai trò lớn trong việc đào tạo độ chính xác. Các phân phối nhãn càng mất cân bằng, việc đào tạo sẽ càng khó khăn hơn. Mặc dù mất entropy chéo có trọng số có thể làm giảm bớt khó khăn, nhưng sự cải thiện không đáng kể cũng như vấn đề nội tại của mất entropy chéo được giải quyết. Trong mất mát entropy chéo, tổn thất được tính bằng mức mất trung bình trên mỗi pixel và tổn thất trên mỗi pixel được tính toán một cách riêng biệt, mà không biết liệu các pixel lân cận của nó có phải là ranh giới hay không. Kết quả là, mất entropy chéo chỉ xem xét mất theo nghĩa vi mô chứ không xem xét nó trên toàn cầu, điều này không đủ để dự đoán mức độ hình ảnh.

* Dice loss

Nó được dựa theo hệ số Sørensen–Dice coefficient, được sử dụng trong thông kê giúp việc xác địn sự tương đồng hai mẫu vào năm 40 của thế kỉ XX. Sau đó vào năm 2016 nó được sử dụng cho phân đoạn y tế  for 3D medical image segmentation.

![](https://miro.medium.com/max/257/1*EF3VCtk-VbTIKhriaQF0YQ.png)

Trong đó pi và gi tương ứng là các cặp giá trị pixel tương ứng của dự đoán và giá trị thực tế.

Trong kịch bản phát hiện ranh giới, các giá trị của pi và gi là 0 hoặc 1, đại diện cho việc pixel có phải là ranh giới (giá trị 1) hay không (giá trị 0). Do đó, mẫu số là tổng của các pixel biên của cả dự đoán và giá trị thực tế( label) và tử số là tổng của các pixel biên được dự đoán chính xác bởi vì tổng chỉ tăng khi pi và gi khớp với nhau (cả hai giá trị 1).

Công thức trên về mặt hình học còn được biểu diễn

![](https://miro.medium.com/max/1400/1*oK8npb1wtF-GKeHi7yIBoQ.png)

DL cũng có thể được định nghĩa băng công thức: ( đây là một biến thể khác tuy nhiên có những biến thể khác nữa)

$\text{DL}\left(p, \hat{p}\right) = 1 - \frac{2p\hat{p} + 1}{p + \hat{p} + 1}$  

Trong đó p là nhãn, $\hat{p}$ là đầu ra dự đoán.

Tại sao mẫu số + 1, do khi p =  $\hat{p}$ = 0 thì mẫu số là 0 


In [ ]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))

    return 1 - numerator / denominator

* Intersection over Union (IoU)-balanced Loss 

$\text{IoU} = \frac{TP}{TP + FP + FN} = \frac{|X \cap Y|}{|X| + |Y| - |X \cap Y|}$

![](https://1.bp.blogspot.com/-P7M5fwU7joM/We_7A3cCqZI/AAAAAAAAAJc/LlVAAsf4iJkKfmejr1Yld7Mo50TAlMjJQCLcBGAs/s1600/iou_equation.png)

Như vậy trong bài toán segment thì chúng ta có thể dựa số pixel được phân loại vào đối tượng với pixel chính xác của đối tượng để tính toán IoU. 

In [ ]:
def IoU_loss(y_true, y_pred):
    numerator = tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
    denominator = tf.reduce_sum(y_true + y_pred - y_true * y_pred, axis=(1,2,3))

    return 1 - numerator / denominator

* So sánh Dice và IoU
Về cơ bản hai hàm loss này cùng tư tưởng tuy nhiên cách triển khai khác đi một chút

$\text{DC} = \frac{2 TP}{2 TP + FP + FN} = \frac{2|X \cap Y|}{|X| + |Y|}$

$\text{IoU} = \frac{TP}{TP + FP + FN} = \frac{|X \cap Y|}{|X| + |Y| - |X \cap Y|}$

Từ đây ta thấy DC > IoU

Tại sao 2 hàm đều là 1 - chỉ số IoU hoặc Dice. Lí do chỉ số kia cho biết sự chính xác dữ liệu phần chồng chéo với tổng thể. 
Nên tính loss tức là phần chưa chính xác. 


* Tversky loss

Là trường hợp tổng quát hóa của Dice loss bằng thêm trọng số vào false positives(FP) và false negatives(FN)

Công thức tính toán: 

$\text{TI}\left(p, \hat{p}\right) = \frac{p\hat{p}}{p\hat{p} + \beta(1 - p)\hat{p} + (1 - \beta)p(1 - \hat{p})})$

Dice loss là th riêng của TL với beta = 1/2 



In [ ]:
def tversky_loss(beta):
    def loss(y_true, y_pred):
        numerator = tf.reduce_sum(y_true * y_pred, axis=-1)
        denominator = y_true * y_pred + beta * (1 - y_true) * y_pred + (1 - beta) * y_true * (1 - y_pred)
        return 1 - (numerator + 1) / (tf.reduce_sum(denominator, axis=-1) + 1)

    return loss

* Boundary loss 

* Lovász-Softmax loss



# KẾT HỢP CÁC HÀM LOSS

Chúng ta cũng có thể kết hợp nhiều hàm loss với nhau đểu cho kết quả tốt hơn

$\text{CE}\left(p, \hat{p}\right) + \text{DL}\left(p, \hat{p}\right)$

In [ ]:
def loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
        denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))

        return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)